<a href="https://colab.research.google.com/github/Jaimemosg/DS4A_Test/blob/master/Ventas_teate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploratory Data Analysis - Teaté

## Load required packages

In [ ]:
import pandas as pd
import re
import io
import os
from unicodedata import normalize
from sqlalchemy import create_engine
from os import listdir
from os.path import isfile, join
import glob

def formatcolumn(text,flag = True):
    text = text.lower()
    text = re.sub('<.*?>', '', text)
    text = re.sub(':.*?:', '', text)
    text = re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD", text), 0, re.I)
    text = normalize( 'NFC', text)
    if flag:
        text = re.sub('[^a-z0-9 ]', '', text)
    else:
        text = re.sub('[^a-z0-9]', '', text)
    return text

#host = 'bootcamprds.cejonyen7g1m.us-east-2.rds.amazonaws.com'
#port = 5432
#user = 'postgres'
#password = '12345678*DS4A'
#database = 'postgres'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

path = r'/content/drive/My Drive/DS4A_T8/archivostat/'
all_files = glob.glob(os.path.join(path, "*.csv"))

df_from_each_file = (pd.read_csv(f,engine='python',sep=';', encoding='latin-1') for f in all_files)
concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)

#concatenated_df = concatenated_df.drop(concatenated_df.columns[[52,53,54,55]], axis=1)
concatenated_df.columns = [formatcolumn(val,False) for val in concatenated_df.columns]
concatenated_df.head()


,pedido,fechapedido,tienda,nombretienda,direcciontienda,fabricante,nombrefabricante,posped,material,nombrematerial,cantidaddepedido,um,valorunitariopedido,mon,valortotalpedido,mon1,valortotalpeddctoiva,mon2,entrega,factura,fechafactura,posfac,ctdfacturada,um1,um2,valorunitariofactura,mon3,valortotalfactura,mon4,descuentoteate,mon5,mr,denominacionmotivorechazo,fechamovil,horamovil,valoringresopedido,mon6,valoringresofactur,mon7,ce,ntatmovil,cupondscto,valorcuponteate,mon8,fechacreaciontienda,mes,unnamed46,unnamed47,unnamed48,zona,nombrezonacomercial,ruta,nombreruta,comuna,barrio,poblacion
0,1000092736,2/01/2020,20009430,Tienda de memo,Cr 58 # 48 sur 15,5000051,DETERGENTES LTDA.,101,861,Detergente Dersa Liquido Doy Pack,6,UN,2.95,COP,17.700,COP,17.700,COP,NaN,NaN,NaN,0,"0,000",NaN,NaN,0.0,COP,0,COP,"0,00",NaN,4.0,No hay stock suficiente,2/01/2020,8:16:24 a. m.,14.874,COP,"0,00",NaN,3000,92108,MENE0120,"0,00",NaN,27/12/2019,Enero,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000092736,2/01/2020,20009430,Tienda de memo,Cr 58 # 48 sur 15,5000039,JOHNSON & JOHNSON DE COLOMBIA S.A,602,382,JOHNSONS baby aceite original 50 ML,1,UN,0.00,COP,0,COP,0,COP,NaN,NaN,NaN,0,"0,000",NaN,NaN,0.0,COP,0,COP,"0,00",NaN,9.0,Dirección Incorrecta,2/01/2020,8:16:24 a. m.,0,COP,"0,00",NaN,3000,92108,MENE0120,"0,00",NaN,27/12/2019,Enero,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000092736,2/01/2020,20009430,Tienda de memo,Cr 58 # 48 sur 15,5000077,PRODUCTOS YUPI S.A.S,302,908,Palomitas Caramelo 68 gr,6,UN,0.00,COP,0,COP,0,COP,NaN,NaN,NaN,0,"0,000",NaN,NaN,0.0,COP,0,COP,"0,00",NaN,9.0,Dirección Incorrecta,2/01/2020,8:16:24 a. m.,0,COP,"0,00",NaN,3000,92108,MENE0120,"0,00",NaN,27/12/2019,Enero,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000092736,2/01/2020,20009430,Tienda de memo,Cr 58 # 48 sur 15,5000079,PROCTER & GAMBLE COLOMBIA LTDA,1002,925,Detergente Rindex 10 250gr,3,UN,0.00,COP,0,COP,0,COP,NaN,NaN,NaN,0,"0,000",NaN,NaN,0.0,COP,0,COP,"0,00",NaN,9.0,Dirección Incorrecta,2/01/2020,8:16:24 a. m.,0,COP,"0,00",NaN,3000,92108,MENE0120,"0,00",NaN,27/12/2019,Enero,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000092736,2/01/2020,20009430,Tienda de memo,Cr 58 # 48 sur 15,5000039,JOHNSON & JOHNSON DE COLOMBIA S.A,601,382,JOHNSONS baby aceite original 50 ML,4,UN,3.50,COP,14.000,COP,14.000,COP,NaN,NaN,NaN,0,"0,000",NaN,NaN,0.0,COP,0,COP,"0,00",NaN,9.0,Dirección Incorrecta,2/01/2020,8:16:24 a. m.,11.765,COP,"0,00",NaN,3000,92108,MENE0120,"0,00",NaN,27/12/2019,Enero,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# concatenated_df.groupby('barrio').count()

In [ ]:
concatenated_df.groupby('poblacion').count()[['pedido']]

,pedido
poblacion,
Bello,29
CALI,3
Caldas,1494
Cali,21
Candelaria,1575
Envigado,3358
Estrella,512
Itagui,844
Itaguí,2104


In [ ]:
connDB = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
conn = connDB.raw_connection()
cur = conn.cursor()

concatenated_df.head(0).to_sql('Ventas', con = connDB, if_exists='replace',index=False) 
concatenated_df = concatenated_df.replace('\n','', regex=True)
concatenated_df = concatenated_df.replace('\r','', regex=True)

output = io.StringIO()
concatenated_df.to_csv(output, sep='|', header=False, index=False)
output.seek(0)
contents = output.getvalue()
cur.copy_from(output, 'Ventas',sep='|') 
conn.commit()
conn.close()
print("Upload Finished")

UndefinedTable: relation "ventas" does not exist
